In [1]:
import os
import json
import pandas as pd
import numpy as np
import cupy as cp
from tqdm import tqdm
from scipy.ndimage import median_filter

class KeypointsDataset:
    def __init__(self, json_dir, output_dir, csv_folders, batch_size=200000):
        self.json_dir = json_dir
        self.output_dir = output_dir
        self.csv_folders = csv_folders
        self.json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]
        self.batch_size = batch_size

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

    def keypoint_to_face_part(self, index):
        if 1 <= index <= 33:
            return 0  # Do not load in df
        elif 34 <= index <= 42:
            return "Right_brow"
        elif 43 <= index <= 51:
            return "Left_brow"
        elif 52 <= index <= 66:
            if 56 <= index <= 66:
                return 0  # Do not load in df
            return "Nose"
        elif 67 <= index <= 75:
            return "Right_Eye"
        elif 76 <= index <= 84:
            return "Left_Eye"
        elif 85 <= index <= 104:
            return "Mouth"
        elif index == 105 or index == 106:
            return 0  # Do not load in df

    def apply_filters(self, df):
        # Apply mean filter across frames for each keypoint index
        df['mean_keypoint_x'] = df.groupby('keypoint_index')['keypoint_x'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
        df['mean_keypoint_y'] = df.groupby('keypoint_index')['keypoint_y'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
        
        # Apply median filter on the mean-filtered data across frames for each keypoint index
        df['median_keypoint_x'] = df.groupby('keypoint_index')['mean_keypoint_x'].transform(lambda x: median_filter(x, size=3))
        df['median_keypoint_y'] = df.groupby('keypoint_index')['mean_keypoint_y'].transform(lambda x: median_filter(x, size=3))
        
        return df

    def move_and_rotate_keypoints_updated(self, df):
        def get_rotation_matrix(angle):
            return np.array([
                [np.cos(angle), -np.sin(angle)],
                [np.sin(angle), np.cos(angle)]
            ])

        def rotate_points(points, angle):
            rotation_matrix = get_rotation_matrix(angle)
            return np.dot(points, rotation_matrix.T)

        def normalize_points(points):
            min_vals = points.min(axis=0)
            max_vals = points.max(axis=0)
            scale = max(max_vals - min_vals)
            normalized_points = 2 * ((points - min_vals) / scale) - 1
            return normalized_points

        def sum_abs_x(angle):
            rotated = rotate_points(translated_coords_np, angle)
            return np.sum(np.abs(rotated[:, 0]))

        result = []

        # keypoint indexes to be aligned
        align_keypoints = [52, 53, 54, 55, 88, 94]

        grouped = df.groupby(['infant_id', 'frame_id'])
        
        for (infant_id, frame_id), group in tqdm(grouped, desc="Processing frames"):
            if group[group['keypoint_index'] == 55].empty:
                continue
            
            kp_54 = cp.array(group[group['keypoint_index'] == 55][['x_coordinate', 'y_coordinate']].values[0])
            
            # coordinates of the keypoints to be aligned
            keypoints_to_align = cp.array(group[group['keypoint_index'].isin(align_keypoints)][['x_coordinate', 'y_coordinate']].values)
            translated_coords = keypoints_to_align - kp_54
            translated_coords_np = translated_coords.get()
            
            # optimal angle to minimize the sum of absolute x-coordinates
            angles = np.linspace(-np.pi, np.pi, 360)
            optimal_angle = angles[np.argmin([sum_abs_x(a) for a in angles])]
            
            # translate all keypoints relative to kp_54
            all_translated_coords = cp.array(group[['x_coordinate', 'y_coordinate']].values) - kp_54
        
            all_translated_coords_np = all_translated_coords.get()
            rotated_coords = rotate_points(all_translated_coords_np, optimal_angle)
            normalized_coords = normalize_points(rotated_coords)

            kp_52_y = normalized_coords[group['keypoint_index'].values == 52][0, 1]
            
            if kp_52_y < 0:
                normalized_coords = rotate_points(normalized_coords, np.pi)

            transformed_group = group.copy()
            transformed_group[['x_coordinate', 'y_coordinate']] = normalized_coords
            
            result.append(transformed_group)
        
        if not result:
            return pd.DataFrame()  
    
        return pd.concat(result).reset_index(drop=True)


    def calculate_features(self, df):
        def mean_y_coordinate(indices, mask):
            return cp.mean(cp_df['y_coordinate'][mask & cp.isin(cp_df['keypoint_index'], indices)])

        def mean_distance(pairs, mask):
            distances = []
            for (p1, p2) in pairs:
                mask1 = mask & (cp_df['keypoint_index'] == p1)
                mask2 = mask & (cp_df['keypoint_index'] == p2)
                if cp.any(mask1) and cp.any(mask2):
                    distances.append(cp.abs(cp.mean(cp_df['y_coordinate'][mask1]) - cp.mean(cp_df['y_coordinate'][mask2])))
            return cp.mean(cp.array(distances))

        def diff_x_coordinates(idx1, idx2, mask):
            x1 = cp.mean(cp_df['x_coordinate'][mask & (cp_df['keypoint_index'] == idx1)])
            x2 = cp.mean(cp_df['x_coordinate'][mask & (cp_df['keypoint_index'] == idx2)])
            return cp.abs(x1 - x2)

        def euclidean_distance(idx1, idx2, mask):
            x1 = cp.mean(cp_df['x_coordinate'][mask & (cp_df['keypoint_index'] == idx1)])
            y1 = cp.mean(cp_df['y_coordinate'][mask & (cp_df['keypoint_index'] == idx1)])
            x2 = cp.mean(cp_df['x_coordinate'][mask & (cp_df['keypoint_index'] == idx2)])
            y2 = cp.mean(cp_df['y_coordinate'][mask & (cp_df['keypoint_index'] == idx2)])
            return cp.sqrt((x2 - x1)**2 + (y2 - y1)**2)

        def calculate_curvature_cp(x_coords, y_coords, mask):
            x_coords_filtered = x_coords[mask]
            y_coords_filtered = y_coords[mask]

            if len(x_coords_filtered) < 3 or len(y_coords_filtered) < 3:
                return cp.nan  

            dx = cp.gradient(x_coords_filtered)
            dy = cp.gradient(y_coords_filtered)

            ddx = cp.gradient(dx)
            ddy = cp.gradient(dy)

            curvature_numer = cp.abs(dx * ddy - dy * ddx)
            curvature_denom = cp.power(dx**2 + dy**2, 1.5)
            
            curvature_denom = cp.where(curvature_denom == 0, cp.nan, curvature_denom)
            curvatures = curvature_numer / curvature_denom
            curvatures = cp.nan_to_num(curvatures, nan=0.0, posinf=0.0, neginf=0.0)
            mean_curvature = cp.mean(curvatures)

            return mean_curvature


        left_brow_indices = cp.array(range(34, 43))
        right_brow_indices = cp.array(range(43, 52))
        keypoint_pairs = {
            'right_eye': [(79, 81), (78, 82), (77, 83)],
            'left_eye': [(70, 72), (69, 73), (68, 74)],
            'lips': [(100, 102), (99, 103), (98, 104)]
        }

        indices_dict = {
            'upper_right_brow': cp.array(range(43, 48)),
            'lower_right_brow': cp.array(range(48, 52)),
            'upper_left_brow': cp.array(range(34, 39)),
            'lower_left_brow': cp.array(range(39, 43)),
            'upper_left_eyelid': cp.array(range(67, 72)),
            'lower_left_eyelid': cp.array(range(72, 75)),
            'upper_right_eyelid': cp.array(range(76, 81)),
            'lower_right_eyelid': cp.array(range(81, 84)),
            'upper_outer_lip': cp.array(range(85, 92)),
            'lower_outer_lip': cp.array(range(92, 98)),
            'upper_inner_lip': cp.array(range(98, 102)),
            'lower_inner_lip': cp.array(range(102, 105))
        }

        cp_df = {
            'infant_id': cp.array(df['infant_id'].values),
            'age': cp.array(df['age_weeks'].values),
            'frame_id': cp.array(df['frame_id'].values),
            'keypoint_index': cp.array(df['keypoint_index'].values),
            'x_coordinate': cp.array(df['x_coordinate'].values),
            'y_coordinate': cp.array(df['y_coordinate'].values)
        }

        results = []
        for (infant_id, age, frame_id), group in tqdm(df.groupby(['infant_id', 'age_weeks', 'frame_id']), desc="Processing videos"):
            mask = (cp_df['infant_id'] == infant_id) & (cp_df['age'] == age) & (cp_df['frame_id'] == frame_id)
            curvatures = {f'curvature_{key}': calculate_curvature_cp(cp_df['x_coordinate'], cp_df['y_coordinate'], mask & cp.isin(cp_df['keypoint_index'], indices)).get() for key, indices in indices_dict.items()}
            features = {
                'right_brow_mean_y': mean_y_coordinate(right_brow_indices, mask).get(),
                'left_brow_mean_y': mean_y_coordinate(left_brow_indices, mask).get(),
                'right_eye_distance': mean_distance(keypoint_pairs['right_eye'], mask).get(),
                'left_eye_distance': mean_distance(keypoint_pairs['left_eye'], mask).get(),
                'lips_distance': mean_distance(keypoint_pairs['lips'], mask).get(),
                'left_eye_width': diff_x_coordinates(67, 71, mask).get(),
                'right_eye_width': diff_x_coordinates(76, 80, mask).get(),
                'left_eyebrow_width': diff_x_coordinates(34, 39, mask).get(),
                'right_eyebrow_width': diff_x_coordinates(47, 51, mask).get(),
                'lip_width': diff_x_coordinates(85, 91, mask).get(),
                'left_eye_upper_corner_left_eyebrow_center_dist': euclidean_distance(67, 41, mask).get(),
                'right_eye_upper_corner_and_right_eyebrow_center_dist': euclidean_distance(80, 49, mask).get(),
                'nose_centre_lips_centre_dist': diff_x_coordinates(55, 99, mask).get(),
                'left_eye_lower_corner_lips_left_corner_dist': euclidean_distance(85, 71, mask).get(),
                'right_eye_lower_corner_lips_right_corner_dist': euclidean_distance(76, 91, mask).get(),
                'left_eye_upper_corner_lips_left_corner_dist': euclidean_distance(85, 67, mask).get(),
                'right_eye_upper_corner_lips_right_corner_dist': euclidean_distance(80, 91, mask).get(),
                **curvatures
            }

            results.append({
                'infant_id': infant_id,
                'age': age,
                'frame_id': frame_id,
                **features
            })

        if not results:
            return pd.DataFrame()  # Return an empty DataFrame if no results are generated
        
        results_df = pd.DataFrame(results)
        return results_df

    def save_to_csv(self, df, filename):
        if df.empty:
            print(f"No data to save for {filename}")
            return

        output_path = os.path.join(self.output_dir, filename)
        df.to_csv(output_path, index=False)
        print(f"Dataset saved successfully as {output_path}")


    def process_and_save(self, json_file):
        video_id = os.path.splitext(json_file)[0]
        parts = video_id.split('_')
        
        infant_id = parts[0]
        if parts[6] == 'week':
            age = int(parts[4])  
        else :
            age = int(parts[4]) * 4

        # Check if the CSV file already exists in any of the csv_folders
        for folder in self.csv_folders:
            if os.path.exists(os.path.join(folder, f"{video_id}.csv")):
                print(f"Skipping {json_file} as {video_id}.csv already exists in {folder}")
                return

        with open(os.path.join(self.json_dir, json_file), 'r') as f:
            frames = json.load(f)
            frame_data_list = []

            for frame_index, frame_data in enumerate(frames):
                frame_id = frame_data["frame_id"]
                for instance_index, instance in enumerate(frame_data["instances"]):
                    keypoints = instance["keypoints"]
                    keypoint_scores = instance["keypoint_scores"]

                    for idx, (kp, score) in enumerate(zip(keypoints, keypoint_scores)):
                        face_part = self.keypoint_to_face_part(idx + 1)
                        if face_part != 0:
                            frame_data_list.append({
                                "infant_id": int(infant_id),
                                "age_weeks": age,
                                "frame_id": frame_id,
                                "keypoint": kp,
                                "keypoint_score": score,
                                "face_part": face_part,
                                "keypoint_index": idx + 1,
                                "keypoint_x": kp[0],
                                "keypoint_y": kp[1]
                            })

            df = pd.DataFrame(frame_data_list)
            if df.empty:
                print(f"No valid data in {json_file}")
                return

            df = self.apply_filters(df)
            df[['x_coordinate', 'y_coordinate']] = pd.DataFrame(df['keypoint'].tolist(), index=df.index)
            df = df.drop(columns=['keypoint'])
    
            # Filter by average confidence over 0.85
            df['average_confidence'] = df.groupby('frame_id')['keypoint_score'].transform('mean')
            df = df[df['average_confidence'] >= 0.85]

            if df.empty:
                print(f"No frames with average confidence over 0.85 in {json_file}")
            else:
                # Identify consecutive frames blocks with at least 20 frames
                df = df.sort_values(by=['keypoint_index','frame_id'])
                df = df.reset_index(drop=True)
                df['frame_diff'] = df['frame_id'].diff().fillna(1)
                df['block'] = (df['frame_diff'] != 1).cumsum()
                blocks = df.groupby('block').filter(lambda x: len(x) >= 20)
                df = blocks.drop(columns=['block', 'frame_diff'])

            if df.empty:
                print(f"No blocks of at least 20 consecutive frames with average confidence over 0.85 in {json_file}")
                return

            df = self.move_and_rotate_keypoints_updated(df)
            if df.empty:
                print(f"No valid keypoints to process in {json_file}")
                return

            df = self.calculate_features(df)
            if df.empty:
                print(f"No features to save for {json_file}")
                return

            self.save_to_csv(df, f"{video_id}.csv")

    def load_data(self):
        for json_file in self.json_files:
            self.process_and_save(json_file)
        print("All files processed and saved.")

# Usage
json_dir = '/workspaces/wiggle-face/data-ioana/PANDA2/face/annotations'
output_dir = '/workspaces/wiggle-face/data/panda_each_video/panda2_features_complete'
csv_folders = [
    #'/workspaces/wiggle-face/data/panda_each_video/panda1_features_complete',
    '/workspaces/wiggle-face/data/panda_each_video/panda2_features_complete'
    #'/workspaces/wiggle-face/data/panda_each_video/panda3_features_complete'
]
dataset = KeypointsDataset(json_dir, output_dir, csv_folders)
dataset.load_data()

Skipping 624_1__age_3_4_month_video.json as 624_1__age_3_4_month_video.csv already exists in /workspaces/wiggle-face/data/panda_each_video/panda2_features_complete
Skipping 97338_1__age_36_42_week_video.json as 97338_1__age_36_42_week_video.csv already exists in /workspaces/wiggle-face/data/panda_each_video/panda2_features_complete
No frames with average confidence over 0.85 in 300_1__age_3_4_month_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 300_1__age_3_4_month_video.json
No frames with average confidence over 0.85 in 59131_2__age_3_4_month_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 59131_2__age_3_4_month_video.json
Skipping 21935_1__age_36_42_week_video.json as 21935_1__age_36_42_week_video.csv already exists in /workspaces/wiggle-face/data/panda_each_video/panda2_features_complete
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 1180_2__age_36_42_week_vide

Processing videos: 100%|██████████| 546/546 [00:13<00:00, 40.28it/s]


Dataset saved successfully as /workspaces/wiggle-face/data/panda_each_video/panda2_features_complete/64_3__age_3_4_month_video.csv
No frames with average confidence over 0.85 in 93243_1__age_36_42_week_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 93243_1__age_36_42_week_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 61993_1__age_36_42_week_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 1325_1__age_3_4_month_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 352_1__age_3_4_month_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 53395_2__age_36_42_week_video.json
No frames with average confidence over 0.85 in 55366_1__age_3_4_month_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 55366_1__age_3_4_month_video.json
No blocks of at 

Processing videos: 100%|██████████| 1839/1839 [01:05<00:00, 27.92it/s]


Dataset saved successfully as /workspaces/wiggle-face/data/panda_each_video/panda2_features_complete/570_1__age_3_4_month_video.csv
No frames with average confidence over 0.85 in 30402_2__age_36_42_week_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 30402_2__age_36_42_week_video.json


Processing videos: 100%|██████████| 587/587 [00:17<00:00, 32.98it/s]


Dataset saved successfully as /workspaces/wiggle-face/data/panda_each_video/panda2_features_complete/467_1__age_3_4_month_video.csv
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 1023_1__age_3_4_month_video.json
No frames with average confidence over 0.85 in 143_1__age_3_4_month_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 143_1__age_3_4_month_video.json


Processing videos: 100%|██████████| 261/261 [00:06<00:00, 38.73it/s]


Dataset saved successfully as /workspaces/wiggle-face/data/panda_each_video/panda2_features_complete/148_4__age_3_4_month_video.csv
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 526_2__age_3_4_month_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 245_1__age_3_4_month_video.json
No frames with average confidence over 0.85 in 49389_1__age_36_42_week_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 49389_1__age_36_42_week_video.json
No frames with average confidence over 0.85 in 1122_1__age_36_42_week_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 1122_1__age_36_42_week_video.json
No frames with average confidence over 0.85 in 12824_1__age_36_42_week_video.json
No blocks of at least 20 consecutive frames with average confidence over 0.85 in 12824_1__age_36_42_week_video.json
All files processed and saved.
